In [55]:
#Funktion, um Benachrichtigung zu schicken

def send_mail(sender, recipient, subject, message):
    from smtplib import SMTP
    from email.mime.text import MIMEText

#header
    msg = MIMEText(message)
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient
    msg['Subject'] = subject
    
#Verbindung zum Server
    server = SMTP('smtp.web.de', 587)
    server.ehlo()
    server.starttls()
    server.login(sender, 'password')
    server.sendmail(sender, recipient, msg.as_string())
    server.quit()



In [11]:
#Funktion, mit dem Messageteil

def send_alert(url, e, errtype):
    subject = "{} for {}".format(errtype, url)
    message = str(e)
    send_mail('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', subject, message,
              )
    


In [12]:
#Link checking ohne csv-Datei
import requests

urls = [
	"https://api.github.com/invalid",
	"https://www.gutenberg.org/",
    "https://www.heise-regioconcept.de/homepages/invalid"
]
#try except Block zum Link checken
for url in urls:
    try:
        r = requests.head(url)
        r.raise_for_status()
    # Want to ensure that any HTTP error gets handled as an exception
    except requests.exceptions.HTTPError as e:
        send_alert(url, e, 'HTTPError')
    # Catch the base class exception to broadly handle all exceptions
    except requests.exceptions.RequestException as e:
        send_alert(url, e, 'Exception')

In [8]:
#E-Mail ohne csv, dass alles OK ist:
def all_ok(sender, recipient):

    from email.message import EmailMessage
    import smtplib

#Header
    msg = EmailMessage()
    msg["Subject"] = "All links are ok - EOM"
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht
    msg.set_content(" ")


# Verbindung
    with smtplib.SMTP('smtp.web.de', 587) as s:
        s.ehlo()
        s.starttls()
        s.login(sender, 'password')
        s.send_message(msg)

In [7]:
#Zusammenfügen von Report erstellen und E-Mail senden:
#Funktion zum E-Mail-Senden:

def send_email_alert(sender, recipient, report):
    
    from email.message import EmailMessage
    import smtplib

#header

    msg = EmailMessage()
    msg["Subject"] = "Broken Links in Reading List"
    #sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    #recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht und Anhang

    msg.set_content("Dear Team, /nsome of the links are not working. Please have a look!")

    with open("broken_links.csv", "rb") as fp:
        msg.add_attachment(
            fp.read(), maintype="broken_links", subtype="csv")
    
#Verbindung

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'password')
        smtplib.send_message(msg)

In [54]:
#Link checking ohne csv-Datei
import requests
import pandas as pd
from datetime import datetime 

#Um Report einen Zeitstempel zu geben:
date = datetime.now().strftime('%Y_%m_%d')

urls = [
	"https://api.github.com/invalid",
	"https://www.gutenberg.org/",
    "https://www.heise-regioconcept.de/homepages/invalid"
]
#try except Block zum Link checken
broken_links = []

for url in urls:
    try:
        response = requests.head(url)
        response.raise_for_status()
        #status = r.status_code
        
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
            #status = http_error.response.status_code
            
            #if status in [400, 404, 500, 503]:    
              broken_links.append(f"{url}: HTTP-Fehlermeldung: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            
             broken_links.append(f"{url}: Fehlermeldung:' {other_error}")
            
pd.Series(list(broken_links)).to_csv(f"{date}_broken_links.csv")
    
    
#send_alert(broken_links)

In [15]:
#Funktion, um email mit csv-Datei zu schicken:

def send_alert(broken_links):
    subject = "Broken Links in Reading List"
    send_email('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', subject, 
              )

In [42]:
broken_links_list = pd.read_csv("broken_links.csv")

In [43]:
broken_links_list

,Unnamed: 0,0
0,0,"https://api.github.com/invalid,{http_error}"
1,1,https://www.gutenberg.org/: Other error occurr...
2,2,https://www.heise-regioconcept.de/homepages/in...


In [1]:
#Link checking mit csv-Datei:

#Importieren aller notwendigen Libraries:
import csv
import pandas as pd
import requests
from email.message import EmailMessage
import smtplib

In [10]:
#Einlesen der csv-Datei mit Pandas:

reading_list = pd.read_csv("~/2021-11-29-DL_Claudia_Wordsworth/Modul_6_Abschlussarbeit/Readinglist_2.csv")

In [3]:
#isolieren der einzulesenden Spalte:
links = reading_list["Link"]
links

0    https://www.heise.de/invalid
1      https://www.gutenberg.org/
2          https://www.yahoo.com/
Name: Link, dtype: object

In [8]:
#request link checking und email, dass alle links ok sind:

broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"Other error occurred: {other_error}")
            
if len(broken_links) == 0:
    all_ok('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de')

#Aus der Liste eine Tabelle machen

#df = pd.DataFrame([[links], [broken_links]], columns=["Link", "Error Code"])
pd.Series(list(broken_links)).to_csv("broken_links.csv")


send_email_alert('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', broken_links)

In [18]:
broken_links

['HTTP error occurred: 404 Client Error: Not Found for url: https://www.heise.de/invalid']

#Report mit csv Writer

In [44]:
#Versuch Report mit CSV writer zu schreiben:
header = ["Link", "Error Code"]
with open("broken_links.csv", "w", newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
        

In [47]:
#Versuch Report mit CSV writer
broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"In {url} HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"In {url} Other error occurred:' {other_error}")
            

Report(broken_links)
#Problem, das jedes einzelne Zeichen als eine Spalte angesehen wird.

In [46]:
#Funktion, um Report mit csv zu schreiben:

def Report(broken_links):
    with open("broken_links.csv", "a", newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(broken_links)
        

In [11]:
reading_list


,Date Added,Title,Type,Item Link,Local Control Number,ISBN10,ISBN13,ISSN,EISSN,DOI,Importance,Note for Student,Note for Library
0,23/05/2022 09:02,Sex and Crime,Book,http://readinglists.mdx.ac.uk/items/0ad62456-3...,9.910030e+17,NaN,9.78153E+12,NaN,NaN,NaN,Essential – PeT,NaN,NaN
1,23/05/2022 09:06,Violent crime: clinical and social implications,Book,http://readinglists.mdx.ac.uk/items/f0dc4f88-2...,9.910020e+17,NaN,9.78141E+12,NaN,NaN,NaN,Recommended,NaN,NaN
2,23/05/2022 09:07,International handbook of violence research,Book,http://readinglists.mdx.ac.uk/items/7339d6c2-6...,9.910020e+17,NaN,9.7814E+12,NaN,NaN,NaN,Recommended,NaN,NaN
3,23/05/2022 09:13,Understanding Violent Crime,Book,http://readinglists.mdx.ac.uk/items/e6f81ea5-d...,9.910010e+17,NaN,9.78034E+12,NaN,NaN,NaN,Recommended,NaN,NaN
4,23/05/2022 09:15,Women and the criminal justice system: failing...,Book,http://readinglists.mdx.ac.uk/items/32f65c49-0...,9.910030e+17,NaN,9.78332E+12,NaN,NaN,NaN,Recommended,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,17/06/2022 11:47,Police Interviews of Sexual Assault Reporters:...,Article,http://readinglists.mdx.ac.uk/items/390d07a2-4...,NaN,NaN,NaN,0886-6708,1945-7073,10.1891/0886-6708.27.2.263,NaN,NaN,NaN
251,17/06/2022 11:48,Comparing Rape Victim and Perpetrator Blaming ...,Article,http://readinglists.mdx.ac.uk/items/29c8e7c9-f...,NaN,NaN,NaN,0093-8548,1552-3594,10.1177/0093854811434696,NaN,NaN,NaN
252,17/06/2022 11:50,Police Officer Schema of Sexual Assault Reports,Article,http://readinglists.mdx.ac.uk/items/f73cb0f1-7...,NaN,NaN,NaN,0886-2605,1552-6518,10.1177/0886260514556765,NaN,NaN,NaN
253,17/06/2022 11:52,Making Judgments: How Blame Mediates the Influ...,Article,http://readinglists.mdx.ac.uk/items/0c299b1f-a...,NaN,NaN,NaN,0886-2605,1552-6518,10.1177/0886260516662437,NaN,NaN,NaN
